<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Meta/blob/master/tutorials/1-Introduction/China_A_share_market_tushare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Quantitative trading in China A stock market with FinRL

Install FinRL

In [1]:
#!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


Install other libraries

In [2]:
# !pip install stockstats
# !pip install tushare
# #install talib
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 
# !tar xvzf ta-lib-0.4.0-src.tar.gz
# import os
# os.chdir('ta-lib') 
# !./configure --prefix=/usr
# !make
# !make install
# #!sudo make install # Sometimes it need root 
# os.chdir('../')
# !pip install TA-Lib

In [3]:
# %cd /
# !git clone https://github.com/AI4Finance-Foundation/FinRL-Meta
# %cd /FinRL-Meta/

##Import Modules

In [1]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
from IPython import display

display.set_matplotlib_formats("svg")

import os
from argparse import ArgumentParser
from typing import List

import pyfolio
from pyfolio import timeseries

from agents.stablebaselines3_models import DRLAgent
from main import check_and_make_directories
from meta import config
from meta.config import (ALPACA_API_BASE_URL, ALPACA_API_KEY,
                         ALPACA_API_SECRET, DATA_SAVE_DIR, ERL_PARAMS,
                         INDICATORS, RESULTS_DIR, SAC_PARAMS,
                         TENSORBOARD_LOG_DIR, TEST_END_DATE, TEST_START_DATE,
                         TRADE_END_DATE, TRADE_START_DATE, TRAIN_END_DATE,
                         TRAIN_START_DATE, TRAINED_MODEL_DIR, RLlib_PARAMS)
from meta.config_tickers import DOW_30_TICKER
from meta.data_processor import DataProcessor
from meta.data_processors.tushare import ReturnPlotter, Tushare
from meta.data_processors.tushare_private import ReturnPlotterPrivate
from meta.env_stock_trading.env_stocktrading_China_A_shares import \
    StockTradingEnv

pd.options.display.max_columns = None

print("ALL Modules have been imported!")

ALL Modules have been imported!


##Create Folders

In [2]:
import os

''' 
use check_and_make_directories() to replace the following

if not os.path.exists("./datasets"): 
  os.makedirs("./datasets") 
if not os.path.exists("./trained_models"): 
  os.makedirs("./trained_models") 
if not os.path.exists("./tensorboard_log"): 
  os.makedirs("./tensorboard_log") 
if not os.path.exists("./results"): 
  os.makedirs("./results") 
'''

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

##Download data, cleaning and feature engineering

In [3]:
import importlib

import meta
from meta.local.selector.stock_selector import StockSelector

importlib.reload(meta.local.selector.stock_selector)

stock_sel = StockSelector()
ticker_list = stock_sel.select_stocks()
print(len(ticker_list))

TRAIN_START_DATE = '2015-01-01' 
TRAIN_END_DATE= '2022-05-01'  
TRADE_START_DATE = '2022-05-01' 
TRADE_END_DATE = '2022-10-27'

TIME_INTERVAL = "1d" 
kwargs = {} 
kwargs['token'] = '27080ec403c0218f96f388bca1b1d85329d563c91a43672239619ef5'
p = DataProcessor(data_source='tushare', start_date=TRAIN_START_DATE, end_date=TRADE_END_DATE, time_interval=TIME_INTERVAL, **kwargs)

17
tushare successfully connected


###Download and Clean

In [4]:
p.download_data(ticker_list=ticker_list)

p.clean_data()

100%|██████████| 17/17 [00:10<00:00,  1.68it/s]


Shape of DataFrame:  (23514, 8)
Shape of DataFrame:  (26614, 8)


###Add technical indicator

In [5]:
p.add_technical_indicator(config.INDICATORS) 
p.clean_data()

#print(f"p.dataframe: {p.dataframe}")

tech_indicator_list:  ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
indicator:  macd
indicator:  boll_ub
indicator:  boll_lb
indicator:  rsi_30
indicator:  cci_30
indicator:  dx_30
indicator:  close_30_sma
indicator:  close_60_sma
Succesfully add technical indicators
Shape of DataFrame:  (13706, 17)


##Split training dataset

In [6]:
train = p.data_split(p.dataframe, TRAIN_START_DATE, TRAIN_END_DATE) 

print(f"len(train.tic.unique()): {len(train.tic.unique())}")

len(train.tic.unique()): 14


In [7]:
print(f"train.tic.unique(): {train.tic.unique()}")

train.tic.unique(): ['000651.SZ' '002241.SZ' '002466.SZ' '300750.SZ' '300760.SZ' '600009.SH'
 '600036.SH' '600276.SH' '600438.SH' '600519.SH' '600740.SH' '601088.SH'
 '603259.SH' '603288.SH']


In [8]:
print(f"train.head(): {train.head()}")

train.head():          tic        date  index   open   high    low  close     volume  day  \
0  000651.SZ  2018-10-17  12908  38.80  38.89  37.55  38.45  413250.35  2.0   
0  002241.SZ  2018-10-17  12909   6.85   6.91   6.62   6.83  149395.37  2.0   
0  002466.SZ  2018-10-17  12910  29.70  29.85  26.75  28.83  246876.87  2.0   
0  300750.SZ  2018-10-17  12911  69.77  71.78  69.21  70.00  142321.65  2.0   
0  300760.SZ  2018-10-17  12912  77.30  77.30  77.30  77.30    1865.47  2.0   

       macd    boll_ub    boll_lb      rsi_30       cci_30       dx_30  \
0 -0.164248  40.521444  35.820556   46.431272    10.530756   10.666325   
0 -0.445199   9.199594   6.777406   32.168754  -218.319107   66.894205   
0 -2.192112  41.589158  27.700842   30.938485  -195.338539   45.520852   
0 -0.050405  74.092877  60.518123   52.824871    79.289388    9.808731   
0  0.560798  73.765412  67.477588  100.000000  1000.000000  100.000000   

   close_30_sma  close_60_sma  
0     38.136333     39.785833  
0 

In [9]:
print(f"train.shape: {train.shape}")

train.shape: (12040, 17)


In [10]:
stock_dimension = len(train.tic.unique()) 
state_space = stock_dimension * (len(config.INDICATORS) + 2) + 1 

print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 14, State Space: 141


##Train

In [11]:
env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": True, "hundred_each_trade": True }

e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [12]:
env_train, _ = e_train_gym.get_sb_env() 

print(f"print(type(env_train)): {print(type(env_train))}")

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>
print(type(env_train)): None


###DDPG

In [24]:
agent = DRLAgent(env=env_train) 
DDPG_PARAMS = { "batch_size": 256, "buffer_size": 50000, "learning_rate": 0.0005, "action_noise": "normal", } 
POLICY_KWARGS = dict(net_arch=dict(pi=[64, 64], qf=[400, 300])) 
model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS, policy_kwargs=POLICY_KWARGS)

trained_ddpg = agent.train_model(model=model_ddpg, tb_log_name='ddpg', total_timesteps=10000, n_eval_episodes=1)

{'batch_size': 256, 'buffer_size': 50000, 'learning_rate': 0.0005, 'action_noise': NormalActionNoise(mu=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], sigma=[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1])}
Using cpu device
Logging to tensorboard_log/ddpg/ddpg_12
Episode: 74
day: 859, episode: 74
begin_total_asset: 1000000.00
end_total_asset: 1441096.76
total_reward: 441096.76
total_cost: 15602.93
total_trades: 12025
Sharpe: 0.497
Episode: 75
day: 859, episode: 75
begin_total_asset: 1000000.00
end_total_asset: 1417111.56
total_reward: 417111.56
total_cost: 1710.44
total_trades: 12026
Sharpe: 0.495
Episode: 76
day: 859, episode: 76
begin_total_asset: 1000000.00
end_total_asset: 1151263.00
total_reward: 151263.00
total_cost: 609.00
total_trades: 12026
Sharpe: 0.294
Episode: 77
day: 859, episode: 77
begin_total_asset: 1000000.00
end_total_asset: 1109385.61
total_reward: 109385.61
total_cost: 619.39
total_trades: 12026
Sharpe: 0.255
------------------------------------
| time/     

###A2C

In [25]:
agent = DRLAgent(env=env_train) 
model_a2c = agent.get_model("a2c")

trained_a2c = agent.train_model(model=model_a2c, tb_log_name='a2c', total_timesteps=50000, n_eval_episodes=1)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to tensorboard_log/a2c/a2c_10
-------------------------------------
| time/                 |           |
|    fps                | 137       |
|    iterations         | 100       |
|    time_elapsed       | 3         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -19.8     |
|    explained_variance | 0.0482    |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | 47.9      |
|    reward             | -3.474607 |
|    std                | 0.997     |
|    value_loss         | 39.6      |
-------------------------------------
Episode: 87
day: 859, episode: 87
begin_total_asset: 1000000.00
end_total_asset: 2685123.56
total_reward: 1685123.56
total_cost: 195513.36
total_trades: 12021
Sharpe: 1.200
---------------------------------------
| time/                 |             |
|    fps                | 

##Trade

In [26]:
trade = p.data_split(p.dataframe, TRADE_START_DATE, TRADE_END_DATE) 
env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": False, "hundred_each_trade": True } 
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)

In [27]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=trained_ddpg, environment=e_trade_gym)
# import pickle
# with open('./df_account_value.pickle', 'wb') as handle:
#     pickle.dump(df_account_value, handle, protocol=pickle.HIGHEST_PROTOCOL)


Episode: 2
day: 117, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 806683.72
total_reward: -193316.28
total_cost: 237.28
total_trades: 361
Sharpe: -1.180
hit end!


In [28]:
print(df_account_value)

           date  account_value
0    2022-05-05   1.000000e+06
1    2022-05-06   9.943862e+05
2    2022-05-09   9.886927e+05
3    2022-05-10   1.002569e+06
4    2022-05-11   1.017396e+06
..          ...            ...
113  2022-10-20   8.249397e+05
114  2022-10-21   8.093197e+05
115  2022-10-24   7.807317e+05
116  2022-10-25   7.738837e+05
117  2022-10-26   8.066837e+05

[118 rows x 2 columns]


In [29]:
df_actions.to_csv("action.csv", index=False) 
print(f"df_actions: {df_actions}")

df_actions:             000651.SZ  002241.SZ  002466.SZ  300750.SZ  300760.SZ  600009.SH  \
date                                                                           
2022-05-05       1000          0          0          0          0          0   
2022-05-06       1000          0          0          0          0          0   
2022-05-09       1000          0          0          0          0          0   
2022-05-10       1000          0          0          0          0          0   
2022-05-11       1000          0          0          0          0          0   
...               ...        ...        ...        ...        ...        ...   
2022-10-19          0          0          0          0          0          0   
2022-10-20          0          0          0          0          0          0   
2022-10-21          0          0          0          0          0          0   
2022-10-24          0          0          0          0          0          0   
2022-10-25          0       

##Backtest

###matplotlib inline

In [30]:
import importlib

import meta.data_processors.tushare_private
importlib.reload(meta.data_processors.tushare_private)
from meta.data_processors.tushare_private import ReturnPlotterPrivate, TICKET_TYPE_INDEX, TICKET_TYPE_TICKET

from matplotlib import pyplot as plt

plotter = ReturnPlotterPrivate(df_account_value, trade, TRADE_START_DATE, TRADE_END_DATE)
plotter.plot(figure_filepath="./China_A_share.png")
# plotter.plot()

plt.close('all')

In [31]:
# ticket: SSE 50：000016.SH
baseline_ticket = "000016.SH"
plotter.plot(baseline_ticket, figure_filepath="./China_A_share_vs_{0}.png".format(baseline_ticket), ticket_type=TICKET_TYPE_INDEX)
plt.close('all')

TICKET_TYPE_INDEX


###CSI 300

In [32]:
baseline_df = plotter.get_baseline("399300.SZ", ticket_type=TICKET_TYPE_INDEX)
print("baseline_df ", baseline_df)

TICKET_TYPE_INDEX
baseline_df         ts_code trade_date      close       open       high        low  \
0    399300.SZ   20221027  3631.1448  3667.6645  3685.0837  3631.0639   
1    399300.SZ   20221026  3656.9027  3633.3107  3700.8797  3633.3107   
2    399300.SZ   20221025  3627.4542  3621.9356  3670.7327  3594.7011   
3    399300.SZ   20221024  3633.3733  3727.9150  3762.1580  3619.9876   
4    399300.SZ   20221021  3742.8929  3753.8903  3768.6895  3724.6900   
..         ...        ...        ...        ...        ...        ...   
114  399300.SZ   20220511  3976.4231  3917.9322  4033.3026  3916.5249   
115  399300.SZ   20220510  3919.8684  3820.6278  3932.6823  3808.5699   
116  399300.SZ   20220509  3877.4364  3883.9514  3914.1116  3854.9353   
117  399300.SZ   20220506  3908.8150  3928.8548  3947.2378  3902.0510   
118  399300.SZ   20220505  4010.2102  3995.3533  4039.7000  3988.9312   

     pre_close    change  pct_chg          vol       amount   dt  \
0    3656.9027  -25.7579

In [33]:
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats 
perf_stats_all = perf_func(returns=daily_return, factor_returns=daily_return_base, positions=None, transactions=None, turnover_denom="AGB")
print("==============DRL Strategy Stats===========")
print(f"perf_stats_all: {perf_stats_all}")

==============DRL Strategy Stats===========
perf_stats_all: Annual return         -0.367943
Cumulative returns    -0.193316
Annual volatility      0.342544
Sharpe ratio          -1.180400
Calmar ratio          -1.189368
Stability              0.660855
Max drawdown          -0.309360
Omega ratio            0.815873
Sortino ratio         -1.691086
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.969158
Daily value at risk   -0.044761
Alpha                       NaN
Beta                        NaN
dtype: float64


In [34]:
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats
perf_stats_all = perf_func(returns=daily_return_base, factor_returns=daily_return_base, positions=None, transactions=None, turnover_denom="AGB")

print("==============Baseline Strategy Stats===========")

print(f"perf_stats_all: {perf_stats_all}")

==============Baseline Strategy Stats===========
perf_stats_all: Annual return          0.234015
Cumulative returns     0.104393
Annual volatility      0.170323
Sharpe ratio           1.329806
Calmar ratio           1.700856
Stability              0.234650
Max drawdown          -0.137587
Omega ratio            1.235210
Sortino ratio          2.063975
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.289954
Daily value at risk   -0.020560
Alpha                  0.000000
Beta                   1.000000
dtype: float64
